In [166]:
# Importing the desired packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)# To see all the columns of a dataframe
pd.set_option('display.max_rows', None)

In [167]:
# Function to reduce the memory usage of various Dataframes
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
       
        1. Iterate over every column
        2. Determine if the column is numeric
        3. Determine if the column can be represented by an integer
        4. Find the min and the max value
        5. Determine and apply the smallest datatype that can fit the range of values

    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [168]:
def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [128]:
# Reading the Previous_application dataset 
prev_app = import_data('New_previous_application.csv')

Memory usage of dataframe is 331.31 MB
Memory usage after optimization is: 78.06 MB
Decreased by 76.4%


In [129]:
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,17145.0,SATURDAY,15,Y,1,XAP,Approved,-73,Cash through the bank,XAP,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,607500.0,THURSDAY,11,Y,1,XNA,Approved,-164,XNA,XAP,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,112500.0,TUESDAY,11,Y,1,XNA,Approved,-301,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,450000.0,MONDAY,7,Y,1,XNA,Approved,-512,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,337500.0,THURSDAY,9,Y,1,Repairs,Refused,-781,Cash through the bank,HC,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high


In [131]:
# Calculating no. of previous loan applications for every current applicant
prev_app['PREV_APP_CNT']=prev_app.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

In [132]:
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,PREV_APP_CNT
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,17145.0,SATURDAY,15,Y,1,XAP,Approved,-73,Cash through the bank,XAP,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,3
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,607500.0,THURSDAY,11,Y,1,XNA,Approved,-164,XNA,XAP,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,6
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,112500.0,TUESDAY,11,Y,1,XNA,Approved,-301,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,4
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,450000.0,MONDAY,7,Y,1,XNA,Approved,-512,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,23
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,337500.0,THURSDAY,9,Y,1,Repairs,Refused,-781,Cash through the bank,HC,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,25


In [133]:
#Visualizing the Categorical columns of Prev_app dataframe
prev_app.select_dtypes(include='category').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 15 columns):
NAME_CONTRACT_TYPE             1670214 non-null category
WEEKDAY_APPR_PROCESS_START     1670214 non-null category
FLAG_LAST_APPL_PER_CONTRACT    1670214 non-null category
NAME_CASH_LOAN_PURPOSE         1670214 non-null category
NAME_CONTRACT_STATUS           1670214 non-null category
NAME_PAYMENT_TYPE              1670214 non-null category
CODE_REJECT_REASON             1670214 non-null category
NAME_CLIENT_TYPE               1670214 non-null category
NAME_GOODS_CATEGORY            1670214 non-null category
NAME_PORTFOLIO                 1670214 non-null category
NAME_PRODUCT_TYPE              1670214 non-null category
CHANNEL_TYPE                   1670214 non-null category
NAME_SELLER_INDUSTRY           1670214 non-null category
NAME_YIELD_GROUP               1670214 non-null category
PRODUCT_COMBINATION            1669868 non-null category
dtypes: category(15)
memory usag

In [134]:
# Visualizing the unique values of 'NAME_CASH_LOAN_PURPOSE' column 
prev_app['NAME_CASH_LOAN_PURPOSE'].value_counts()

XAP                                 922661
XNA                                 677918
Repairs                              23765
Other                                15608
Urgent needs                          8412
Buying a used car                     2888
Building a house or an annex          2693
Everyday expenses                     2416
Medicine                              2174
Payments on other loans               1931
Education                             1573
Journey                               1239
Purchase of electronic equipment      1061
Buying a new car                      1012
Wedding / gift / holiday               962
Buying a home                          865
Car repairs                            797
Furniture                              749
Buying a holiday home / land           533
Business development                   426
Gasification / water supply            300
Buying a garage                        136
Hobby                                   55
Money for a

In [135]:
# Reclassifying minority values in 'NAME_CASH_LOAN_PURPOSE' as other
prev_app['NAME_CASH_LOAN_PURPOSE'] = prev_app['NAME_CASH_LOAN_PURPOSE'].astype(str)
prev_app.loc[~(prev_app['NAME_CASH_LOAN_PURPOSE'].isin(['XAP','XNA']))
             ,'NAME_CASH_LOAN_PURPOSE'] = 'other'
prev_app['NAME_CASH_LOAN_PURPOSE'] = prev_app['NAME_CASH_LOAN_PURPOSE'].astype('category')

In [136]:
# Confirming the result
prev_app['NAME_CASH_LOAN_PURPOSE'].value_counts()

XAP      922661
XNA      677918
other     69635
Name: NAME_CASH_LOAN_PURPOSE, dtype: int64

In [137]:
# Reclassifying minority values in 'NAME_CASH_LOAN_PURPOSE' as other
ls = ['XNA','Mobile','Consumer Electronics','Computers','Audio/Video']
prev_app['NAME_GOODS_CATEGORY'] = prev_app['NAME_GOODS_CATEGORY'].astype(str)
prev_app.loc[~(prev_app['NAME_GOODS_CATEGORY'].isin(ls))
             ,'NAME_GOODS_CATEGORY'] = 'other'
prev_app['NAME_GOODS_CATEGORY'] = prev_app['NAME_GOODS_CATEGORY'].astype('category')

In [138]:
# Confirming the result
prev_app['NAME_GOODS_CATEGORY'].value_counts()

XNA                     950809
Mobile                  224708
other                   167911
Consumer Electronics    121576
Computers               105769
Audio/Video              99441
Name: NAME_GOODS_CATEGORY, dtype: int64

In [139]:
# Reclassifying minority values in 'NAME_SELLER_INDUSTRY' as other
ls = ['XNA','Consumer electronics','Connectivity']
prev_app['NAME_SELLER_INDUSTRY'] = prev_app['NAME_SELLER_INDUSTRY'].astype(str)
prev_app.loc[~(prev_app['NAME_SELLER_INDUSTRY'].isin(ls))
             ,'NAME_SELLER_INDUSTRY'] = 'other'
prev_app['NAME_SELLER_INDUSTRY'] = prev_app['NAME_SELLER_INDUSTRY'].astype('category')

In [140]:
# Confirming the result
prev_app['NAME_SELLER_INDUSTRY'].value_counts()

XNA                     855720
Consumer electronics    398265
Connectivity            276029
other                   140200
Name: NAME_SELLER_INDUSTRY, dtype: int64

In [141]:
# Reclassifying minority values in 'PRODUCT_COMBINATION' as other
ls = ['Cash','POS household with interest','POS mobile with interest','Cash X-Sell: middle','Cash X-Sell: low',
     'Card Street','POS industry with interest','POS household without interest','Card X-Sell','Cash Street: high',
     'Cash X-Sell: high']
prev_app['PRODUCT_COMBINATION'] = prev_app['PRODUCT_COMBINATION'].astype(str)
prev_app.loc[~(prev_app['PRODUCT_COMBINATION'].isin(ls))
             ,'PRODUCT_COMBINATION'] = 'other'
prev_app['PRODUCT_COMBINATION'] = prev_app['PRODUCT_COMBINATION'].astype('category')

In [142]:
# Confirming the result
prev_app['PRODUCT_COMBINATION'].value_counts()

Cash                              285990
POS household with interest       263622
POS mobile with interest          220670
Cash X-Sell: middle               143883
other                             131956
Cash X-Sell: low                  130248
Card Street                       112582
POS industry with interest         98833
POS household without interest     82908
Card X-Sell                        80582
Cash Street: high                  59639
Cash X-Sell: high                  59301
Name: PRODUCT_COMBINATION, dtype: int64

In [143]:
# Getting the Dummies for the categorical Columns
prev_app = pd.get_dummies(prev_app)

In [144]:
# Dropping the 'SK_ID_PREV' Column
prev_app.drop(columns=['SK_ID_PREV'],inplace=True)

In [145]:
# Collpsing the prev_app Dataframe
prev_app = prev_app.groupby('SK_ID_CURR',as_index=False).mean()

In [146]:
prev_app.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,PREV_APP_CNT,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_XAP,NAME_CASH_LOAN_PURPOSE_XNA,NAME_CASH_LOAN_PURPOSE_other,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,NAME_PAYMENT_TYPE_Cash through the bank,NAME_PAYMENT_TYPE_Cashless from the account of the employer,NAME_PAYMENT_TYPE_Non-cash from your account,NAME_PAYMENT_TYPE_XNA,CODE_REJECT_REASON_CLIENT,CODE_REJECT_REASON_HC,CODE_REJECT_REASON_LIMIT,CODE_REJECT_REASON_SCO,CODE_REJECT_REASON_SCOFR,CODE_REJECT_REASON_SYSTEM,CODE_REJECT_REASON_VERIF,CODE_REJECT_REASON_XAP,CODE_REJECT_REASON_XNA,NAME_CLIENT_TYPE_New,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_GOODS_CATEGORY_Audio/Video,NAME_GOODS_CATEGORY_Computers,NAME_GOODS_CATEGORY_Consumer Electronics,NAME_GOODS_CATEGORY_Mobile,NAME_GOODS_CATEGORY_XNA,NAME_GOODS_CATEGORY_other,NAME_PORTFOLIO_Cards,NAME_PORTFOLIO_Cars,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell,CHANNEL_TYPE_AP+ (Cash loan),CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_XNA,NAME_SELLER_INDUSTRY_other,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_other
0,100001,3951.000000,24835.50,23787.00,24835.5,13.000000,1.0,-1740.0,23.0,8.0,1,0.000000,1.000000,0.0,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0
1,100002,9251.775391,179055.00,179055.00,179055.0,9.000000,1.0,-606.0,500.0,24.0,1,0.000000,1.000000,0.0,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.000000,1.000000,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.000000,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0
2,100003,56553.988281,435436.50,484191.00,435436.5,14.666667,1.0,-1305.0,533.0,10.0,3,0.333333,0.666667,0.0,0.0,0.333333,0.0,0.333333,0.333333,0.0,0.0,0.0,0.0,1.0,0.666667,0.333333,0.0,1.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,0.333333,0.0,0.0,0.0,0.333333,0.0,0.333333,0.333333,

In [149]:
# 1st List of categorical columns to be dropped
cols_drop1 = ['NAME_CONTRACT_TYPE_XNA','WEEKDAY_APPR_PROCESS_START_SUNDAY','FLAG_LAST_APPL_PER_CONTRACT_N','NAME_CASH_LOAN_PURPOSE_other',
           'NAME_CONTRACT_STATUS_Unused offer','NAME_PAYMENT_TYPE_Non-cash from your account','NAME_PAYMENT_TYPE_Cashless from the account of the employer',
           'CODE_REJECT_REASON_XNA','CODE_REJECT_REASON_VERIF','CODE_REJECT_REASON_SYSTEM','CODE_REJECT_REASON_SCOFR','CODE_REJECT_REASON_SCO',
           'CODE_REJECT_REASON_CLIENT']

In [150]:
# 2nd List of categorical columns to be dropped
cols_drop2=['NAME_GOODS_CATEGORY_other','NAME_CLIENT_TYPE_XNA',
    'NAME_PORTFOLIO_Cars','NAME_PRODUCT_TYPE_walk-in','CHANNEL_TYPE_Car dealer','CHANNEL_TYPE_Contact center',
            'CHANNEL_TYPE_AP+ (Cash loan)','CHANNEL_TYPE_Channel of corporate sales','NAME_SELLER_INDUSTRY_other','NAME_YIELD_GROUP_low_action',
            'PRODUCT_COMBINATION_other']

In [151]:
# Creating the final list of columns to be dropped to remove linearly independent columns
cols_drop = cols_drop1+cols_drop2

In [152]:
# Dropping the undesirable columns
prev_app.drop(columns=cols_drop,inplace=True)

In [78]:
prev_app.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,PREV_APP_CNT,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_XAP,NAME_CASH_LOAN_PURPOSE_XNA,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_PAYMENT_TYPE_Cash through the bank,NAME_PAYMENT_TYPE_XNA,CODE_REJECT_REASON_HC,CODE_REJECT_REASON_LIMIT,CODE_REJECT_REASON_XAP,NAME_CLIENT_TYPE_New,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_GOODS_CATEGORY_Audio/Video,NAME_GOODS_CATEGORY_Computers,NAME_GOODS_CATEGORY_Consumer Electronics,NAME_GOODS_CATEGORY_Mobile,NAME_GOODS_CATEGORY_XNA,NAME_PORTFOLIO_Cards,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_x-sell,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS mobile with interest
0,100001,3951.000000,24835.50,23787.00,24835.5,13.000000,1.0,-1740.0,23.0,8.0,1,0.000000,1.000000,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,1.0,1.000000,0.000000,1.0,0.0,0.0,1.000000,0.000000,0.0,0.0,1.0,0.0,1.000000,0.000000,0.0,0.0,0.000000,1.0,0.000000,0.0,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0
1,100002,9251.775391,179055.00,179055.00,179055.0,9.000000,1.0,-606.0,500.0,24.0,1,0.000000,1.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,1.0,1.000000,0.000000,1.0,0.0,0.0,0.000000,1.000000,0.0,0.0,1.0,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,100003,56553.988281,435436.50,484191.00,435436.5,14.666667,1.0,-1305.0,533.0,10.0,3,0.333333,0.666667,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,1.0,0.666667,0.333333,1.0,0.0,0.0,0.666667,0.333333,0.0,0.0,1.0,0.0,0.666667,0.333333,0.0,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.666667,0.0,0.666667,0.333333,0.333333,0.333333,0.0,0.333333,0.0,0.333333,0.333333,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.0
3,100004,5357.250000,24282.00,20106.00,24282.0,5.000000,1.0,-815.0,30.0,4.0,1,0.000000,1.000000,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,1.0,1.000000,0.000000,1.0,0.0,0.0,1.000000,0.000000,0.0,0.0,1.0,1.0,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.000000,0.0,0.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
4,100005,4813.200195,22308.75,20076.75,44617.5,10.500000,1.0,-536.0,18.0,12.0,2,0.500000,0.500000,0.0,0.500000,0.0,0.000000,0.5,0.0,0.0,1.0,0.500000,0.500000,0.5,0.5,0.0,0.500000,0.500000,0.0,0.0,1.0,0.5,0.000000,0.500000,0.0,0.0,0.000000,0.5,0.500000,0.0,0.000000,0.500000,0.5,1.000000,0.000000,0.500000,0.500000,0.0,0.000000,0.5,0.000000,0.500000,0.5,0.5,0.000000,0.000000,0

In [154]:
# Pre-pending prev_app to each column name of dataframe prev_app
ls = ['prev_app_'+col for col in prev_app.columns if col!='SK_ID_CURR']
ls.insert(0,'SK_ID_CURR')
prev_app.columns=ls

In [155]:
# Confirming the result
prev_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 338857 entries, 0 to 338856
Data columns (total 67 columns):
SK_ID_CURR                                                     338857 non-null int64
prev_app_AMT_ANNUITY                                           338377 non-null float32
prev_app_AMT_APPLICATION                                       338857 non-null float32
prev_app_AMT_CREDIT                                            338857 non-null float32
prev_app_AMT_GOODS_PRICE                                       337793 non-null float32
prev_app_HOUR_APPR_PROCESS_START                               338857 non-null float64
prev_app_NFLAG_LAST_APPL_IN_DAY                                338857 non-null float64
prev_app_DAYS_DECISION                                         338857 non-null float64
prev_app_SELLERPLACE_AREA                                      338857 non-null float64
prev_app_CNT_PAYMENT                                           338379 non-null float16
prev_app_PREV_APP_CNT

In [156]:
#Saving the prev_app dataframe to a csv file.
prev_app.to_csv('Previous_application_merged.csv',index=False)

### Processing  POS_CASH_BALANCE dataset for merging with main application dataset.

In [82]:
pos_cash = import_data('POS_CASH_balance.csv')

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 171.69 MB
Decreased by 71.9%


In [83]:
pos_cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [84]:
pos_cash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
SK_ID_PREV               int32
SK_ID_CURR               int32
MONTHS_BALANCE           int8
CNT_INSTALMENT           float16
CNT_INSTALMENT_FUTURE    float16
NAME_CONTRACT_STATUS     category
SK_DPD                   int16
SK_DPD_DEF               int16
dtypes: category(1), float16(2), int16(2), int32(2), int8(1)
memory usage: 171.7 MB


In [85]:
#Visualizing the unique value of Categorical column
pos_cash['NAME_CONTRACT_STATUS'].value_counts()

Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: NAME_CONTRACT_STATUS, dtype: int64

In [86]:
# Visualizing the unique values of SK_ID_PREV
pos_cash['SK_ID_PREV'].value_counts().head(10)

1624618    96
2746611    96
1889497    96
1235285    96
2263451    96
1835828    96
1000256    96
1856103    96
2687350    96
1012861    96
Name: SK_ID_PREV, dtype: int64

In [87]:
# Visualizing the unique values of SK_ID_CURR
pos_cash['SK_ID_CURR'].value_counts().head(10)

265042    295
172612    247
309133    246
197583    245
127659    245
185185    245
203046    244
362661    239
398407    237
228307    235
Name: SK_ID_CURR, dtype: int64

In [88]:
# Dropping the 'SK_ID_PREV' column as it is redundant over here
pos_cash.drop(columns=['SK_ID_PREV'],inplace=True)

In [89]:
pos_cash.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,182943,-31,48.0,45.0,Active,0,0
1,367990,-33,36.0,35.0,Active,0,0
2,397406,-32,12.0,9.0,Active,0,0
3,269225,-35,48.0,42.0,Active,0,0
4,334279,-35,36.0,35.0,Active,0,0


In [90]:
#getting the Categorical columns of Pos_cash
pos_cash = pd.get_dummies(pos_cash)

In [93]:
# Collaspsing the pos_cash dataset
pos_cash = pos_cash.groupby('SK_ID_CURR',as_index=False).mean()

In [94]:
pos_cash.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,100001,-72.555556,4.000000,1.444336,0.777778,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0
1,100002,-10.000000,24.000000,15.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,100003,-43.785714,10.109375,5.785156,0.000000,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0
3,100004,-25.500000,3.750000,2.250000,0.000000,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0
4,100005,-20.000000,11.703125,7.199219,0.000000,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0


In [96]:
# List of columns to be removed
cols_list = ['NAME_CONTRACT_STATUS_Demand','NAME_CONTRACT_STATUS_Returned to the store',
           'NAME_CONTRACT_STATUS_Approved','NAME_CONTRACT_STATUS_Amortized debt',
           'NAME_CONTRACT_STATUS_Canceled','NAME_CONTRACT_STATUS_XNA']

In [97]:
# Deleting the unwanted columns
pos_cash.drop(columns=cols_list,inplace=True)

In [98]:
pos_cash.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Signed
0,100001,-72.555556,4.000000,1.444336,0.777778,0.777778,0.777778,0.222222,0.000000
1,100002,-10.000000,24.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,100003,-43.785714,10.109375,5.785156,0.000000,0.000000,0.928571,0.071429,0.000000
3,100004,-25.500000,3.750000,2.250000,0.000000,0.000000,0.750000,0.250000,0.000000
4,100005,-20.000000,11.703125,7.199219,0.000000,0.000000,0.818182,0.090909,0.090909


In [157]:
# Pre-pending pos_cash_ to the front of every column of pos_cash dataframe
ls = ['pos_cash_'+col for col in pos_cash.columns if col!='SK_ID_CURR']
ls.insert(0,'SK_ID_CURR')
pos_cash.columns=ls

In [158]:
# Checking the result
pos_cash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337252 entries, 0 to 337251
Data columns (total 9 columns):
SK_ID_CURR                                 337252 non-null int64
pos_cash_MONTHS_BALANCE                    337252 non-null float64
pos_cash_CNT_INSTALMENT                    337224 non-null float16
pos_cash_CNT_INSTALMENT_FUTURE             337224 non-null float16
pos_cash_SK_DPD                            337252 non-null float64
pos_cash_SK_DPD_DEF                        337252 non-null float64
pos_cash_NAME_CONTRACT_STATUS_Active       337252 non-null float64
pos_cash_NAME_CONTRACT_STATUS_Completed    337252 non-null float64
pos_cash_NAME_CONTRACT_STATUS_Signed       337252 non-null float64
dtypes: float16(2), float64(6), int64(1)
memory usage: 21.9 MB


In [159]:
#Saving the pos_cash dataframe to a csv file
pos_cash.to_csv('POS_CASH_balance_merged.csv',index=False)

### Processing  Installment_Payment dataset for merging with main application dataset.

In [103]:
# Reading the file
install_pay=import_data('installments_payments.csv')

Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%


In [104]:
install_pay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
SK_ID_PREV                int32
SK_ID_CURR                int32
NUM_INSTALMENT_VERSION    float16
NUM_INSTALMENT_NUMBER     int16
DAYS_INSTALMENT           float16
DAYS_ENTRY_PAYMENT        float16
AMT_INSTALMENT            float32
AMT_PAYMENT               float32
dtypes: float16(3), float32(2), int16(1), int32(2)
memory usage: 311.4 MB


#### No Categorical columns are present in the dataframe install_pay

In [67]:
# Visualizing the unique values of SK_ID_PREV
install_pay['SK_ID_PREV'].value_counts().head(10)

2360056    293
2592574    279
1017477    248
1449382    243
1746731    236
1690678    223
2709164    222
1383111    220
1152155    219
2543266    216
Name: SK_ID_PREV, dtype: int64

In [68]:
# Visualizing the unique values of SK_ID_CURR
install_pay['SK_ID_CURR'].value_counts().head(10)

145728    372
296205    350
453103    347
189699    344
186851    337
172690    336
418081    332
192083    324
434807    323
217360    318
Name: SK_ID_CURR, dtype: int64

In [105]:
# Dropping the SK_ID_PREV Column
install_pay.drop(columns='SK_ID_PREV',inplace=True)

In [106]:
install_pay.head()

,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [109]:
# Collapsing the install_pay dataframe
install_pay=install_pay.groupby('SK_ID_CURR',as_index=False).mean()

In [110]:
install_pay.head()

,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,100001,1.142578,2.714286,-2188.0,-2196.0,5885.132324,5885.132324
1,100002,1.052734,10.000000,-295.0,-315.5,11559.247070,11559.247070
2,100003,1.040039,5.080000,-1378.0,-1385.0,64754.585938,64754.585938
3,100004,1.333008,2.000000,-754.0,-761.5,7096.154785,7096.154785
4,100005,1.111328,5.000000,-586.0,-609.5,6240.205078,6240.205078


In [160]:
# Pre-pending install_pay_ to the front of every column of install_pay dataframe
ls = ['install_pay_'+col for col in install_pay.columns if col!='SK_ID_CURR']
ls.insert(0,'SK_ID_CURR')
install_pay.columns=ls

In [161]:
# checking the result
install_pay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339587 entries, 0 to 339586
Data columns (total 7 columns):
SK_ID_CURR                            339587 non-null int64
install_pay_NUM_INSTALMENT_VERSION    339587 non-null float16
install_pay_NUM_INSTALMENT_NUMBER     339587 non-null float64
install_pay_DAYS_INSTALMENT           339587 non-null float16
install_pay_DAYS_ENTRY_PAYMENT        339578 non-null float16
install_pay_AMT_INSTALMENT            339587 non-null float32
install_pay_AMT_PAYMENT               339578 non-null float32
dtypes: float16(3), float32(2), float64(1), int64(1)
memory usage: 12.3 MB


In [162]:
# Storing the the install_pay dataframe as csv file
install_pay.to_csv('Installments_payments_merged.csv',index=False)

### Processing  Credit_card_balance dataset for merging with main application dataset.

In [114]:
cc_bal=import_data('credit_card_balance.csv')

Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 263.69 MB
Decreased by 60.9%


In [115]:
cc_bal.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,1800.0,1800.0,0.000000,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,2250.0,2250.0,60175.078125,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,2250.0,2250.0,26926.425781,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,11925.0,11925.0,224949.281250,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,27000.0,27000.0,443044.406250,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [116]:
cc_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
SK_ID_PREV                    int32
SK_ID_CURR                    int32
MONTHS_BALANCE                int8
AMT_BALANCE                   float32
AMT_CREDIT_LIMIT_ACTUAL       int32
AMT_DRAWINGS_ATM_CURRENT      float32
AMT_DRAWINGS_CURRENT          float32
AMT_DRAWINGS_OTHER_CURRENT    float32
AMT_DRAWINGS_POS_CURRENT      float32
AMT_INST_MIN_REGULARITY       float32
AMT_PAYMENT_CURRENT           float32
AMT_PAYMENT_TOTAL_CURRENT     float32
AMT_RECEIVABLE_PRINCIPAL      float32
AMT_RECIVABLE                 float32
AMT_TOTAL_RECEIVABLE          float32
CNT_DRAWINGS_ATM_CURRENT      float16
CNT_DRAWINGS_CURRENT          int16
CNT_DRAWINGS_OTHER_CURRENT    float16
CNT_DRAWINGS_POS_CURRENT      float16
CNT_INSTALMENT_MATURE_CUM     float16
NAME_CONTRACT_STATUS          category
SK_DPD                        int16
SK_DPD_DEF                    int16
dtypes: category(1), float1

In [117]:
## Visualizing the unique values of SK_ID_PREV
cc_bal['SK_ID_PREV'].value_counts().head()

1009171    96
1348858    96
1745395    96
2526035    96
1567893    96
Name: SK_ID_PREV, dtype: int64

In [118]:
## Visualizing the unique values of SK_ID_CURR
cc_bal['SK_ID_CURR'].value_counts().head()

186401    192
311118    178
120076    140
128827    129
246089    128
Name: SK_ID_CURR, dtype: int64

In [119]:
## Geting rid of SK_ID_PREV column
cc_bal.drop(columns='SK_ID_PREV',inplace=True)

In [120]:
# Checking the unique values of the categorical column NAME_CONTRACT_STATUS
cc_bal['NAME_CONTRACT_STATUS'].value_counts()

Active           3698436
Completed         128918
Signed             11058
Demand              1365
Sent proposal        513
Refused               17
Approved               5
Name: NAME_CONTRACT_STATUS, dtype: int64

In [121]:
# Getting the dummies of the categorical columns
cc_bal = pd.get_dummies(cc_bal)

In [122]:
# Collapsing the cc_bal dataframe
cc_bal = cc_bal.groupby('SK_ID_CURR',as_index=False).mean()

In [123]:
cc_bal.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,100006,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,100011,-38.5,54482.109375,164189.189189,2432.432373,2432.432373,0.0,0.0,3956.221924,4843.063965,4520.067383,52402.089844,54433.179688,54433.179688,0.054047,0.054054,0.0,0.0,25.765625,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,100013,-48.5,18159.919922,131718.750000,6350.000000,5953.125000,0.0,0.0,1454.539551,7168.346191,6817.172363,17255.560547,18101.080078,18101.080078,0.255615,0.239583,0.0,0.0,18.718750,0.010417,0.010417,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,100021,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0
4,100023,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [100]:
# listing the columns to drop from the cc_bal dataframe to remove linear independence 
cols_list = ['NAME_CONTRACT_STATUS_Approved','NAME_CONTRACT_STATUS_Demand','NAME_CONTRACT_STATUS_Refused',
          'NAME_CONTRACT_STATUS_Sent proposal','NAME_CONTRACT_STATUS_Signed']

In [101]:
Dropping the columns
cc_bal.drop(columns=cols_list,inplace=True)

In [124]:
cc_bal.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,100006,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,100011,-38.5,54482.109375,164189.189189,2432.432373,2432.432373,0.0,0.0,3956.221924,4843.063965,4520.067383,52402.089844,54433.179688,54433.179688,0.054047,0.054054,0.0,0.0,25.765625,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,100013,-48.5,18159.919922,131718.750000,6350.000000,5953.125000,0.0,0.0,1454.539551,7168.346191,6817.172363,17255.560547,18101.080078,18101.080078,0.255615,0.239583,0.0,0.0,18.718750,0.010417,0.010417,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,100021,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0
4,100023,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [163]:
# Pre-pending cc_bal_ to the front of every column of cc_bal dataframe
ls = ['cc_bal_'+col for col in cc_bal.columns if col!='SK_ID_CURR']
ls.insert(0,'SK_ID_CURR')
cc_bal.columns=ls

In [164]:
# Verifying the result
cc_bal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103558 entries, 0 to 103557
Data columns (total 28 columns):
SK_ID_CURR                                   103558 non-null int64
cc_bal_MONTHS_BALANCE                        103558 non-null float64
cc_bal_AMT_BALANCE                           103558 non-null float32
cc_bal_AMT_CREDIT_LIMIT_ACTUAL               103558 non-null float64
cc_bal_AMT_DRAWINGS_ATM_CURRENT              72194 non-null float32
cc_bal_AMT_DRAWINGS_CURRENT                  103558 non-null float32
cc_bal_AMT_DRAWINGS_OTHER_CURRENT            72194 non-null float32
cc_bal_AMT_DRAWINGS_POS_CURRENT              72194 non-null float32
cc_bal_AMT_INST_MIN_REGULARITY               103558 non-null float32
cc_bal_AMT_PAYMENT_CURRENT                   72120 non-null float32
cc_bal_AMT_PAYMENT_TOTAL_CURRENT             103558 non-null float32
cc_bal_AMT_RECEIVABLE_PRINCIPAL              103558 non-null float32
cc_bal_AMT_RECIVABLE                         103558 non-null float3

In [165]:
#Saving the dataframe cc_bal to a csv file
cc_bal.to_csv('Credit_card_balance_merged.csv',index=False)